Lire Readme
- Crée environnement virtuelle
- Installation les requirments
- Télécharger les données bruit
- Nettoyer les données

In [1]:
import os
import pandas as pd
from utiles import preprocess, analyse

In [2]:
path_data = "data"
path_titanic = os.path.join(path_data, "titanic.csv")
df_bruit = pd.read_csv(path_titanic)

df = preprocess(df_bruit)

Séparer les train test

In [3]:
from sklearn.model_selection import train_test_split

train, res= train_test_split(df, test_size=0.4)
valid, test = train_test_split(res, test_size=0.5)
print(f"nombre données total: {len(df)}")
print(f"nombre train: {len(train)}")
print(f"nombre valid: {len(valid)}")
print(f"nombre test: {len(test)}")

nombre données total: 891
nombre train: 534
nombre valid: 178
nombre test: 179


Définit x_col et y_col

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
y_col = ["Survived"]

In [6]:
# x_col = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
x_col = ['Name', 'Embarked', 'PassengerId']


# model Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier
params_dt = {
    "max_depth": 15,
    "random_state": 42,
}

In [8]:
model = DecisionTreeClassifier(**params_dt)
model.fit(train[x_col], train[y_col])

DecisionTreeClassifier(max_depth=15, random_state=42)

In [9]:
y_pred = model.predict(valid[x_col])
analyse(valid[y_col], y_pred)

TN 75 FP 44 FN 33 TP 26
f1: 0.5753781290444623


In [10]:
def train_dtmodel(x_train, y_train, x_valid, params=params_dt):
    model = DecisionTreeClassifier(**params)
    model.fit(x_train, y_train)
    return model.predict(x_valid)

# model RandomForest

In [11]:
from sklearn.ensemble import RandomForestClassifier
params = {
    "max_depth": 15,
    "n_estimators": 500,
    "max_samples": 0.9,
    "random_state": 42,
}

In [12]:
model = RandomForestClassifier(**params)
model.fit(train[x_col], train[y_col].values.reshape(-1))

RandomForestClassifier(max_depth=15, max_samples=0.9, n_estimators=500,
                       random_state=42)

In [13]:
y_pred = model.predict(valid[x_col])
analyse(valid[y_col], y_pred)

TN 90 FP 29 FN 37 TP 22
f1: 0.621759386133187


In [14]:
def train_rfmodel(x_train, y_train, x_valid, params=params):
    model = RandomForestClassifier(**params)
    model.fit(x_train, y_train.values.reshape(-1))
    return model.predict(x_valid)


# Jouer avec les autres méthode
preprocessing / x_col / model / parmas

In [15]:
def test_col(x_col, params1=params_dt, params2=params):
    print("Decision Tree")
    print("params.", params1)
    print("x_col", x_col)
    y_pred = train_dtmodel(train[x_col], train[y_col], valid[x_col], params1)
    analyse(valid[y_col], y_pred)

    print("---------------")
    print("Random Forest")
    print("params.", params2)
    print("x_col", x_col)
    y_pred = train_rfmodel(train[x_col], train[y_col], valid[x_col], params2)
    analyse(valid[y_col], y_pred)

In [16]:
test_col(x_col)

Decision Tree
params. {'max_depth': 15, 'random_state': 42}
x_col ['Name', 'Embarked', 'PassengerId']
TN 75 FP 44 FN 33 TP 26
f1: 0.5753781290444623
---------------
Random Forest
params. {'max_depth': 15, 'n_estimators': 500, 'max_samples': 0.9, 'random_state': 42}
x_col ['Name', 'Embarked', 'PassengerId']
TN 90 FP 29 FN 37 TP 22
f1: 0.621759386133187


In [17]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [18]:
x_col = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
test_col(x_col)

Decision Tree
params. {'max_depth': 15, 'random_state': 42}
x_col ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
TN 98 FP 21 FN 17 TP 42
f1: 0.7881919353645269
---------------
Random Forest
params. {'max_depth': 15, 'n_estimators': 500, 'max_samples': 0.9, 'random_state': 42}
x_col ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
TN 108 FP 11 FN 16 TP 43
f1: 0.8465192845226652


In [19]:
x_col = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
test_col(x_col)

Decision Tree
params. {'max_depth': 15, 'random_state': 42}
x_col ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
TN 93 FP 26 FN 21 TP 38
f1: 0.7384887230750111
---------------
Random Forest
params. {'max_depth': 15, 'n_estimators': 500, 'max_samples': 0.9, 'random_state': 42}
x_col ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
TN 109 FP 10 FN 16 TP 43
f1: 0.8518159750546009


In [20]:
p1 = {
    "max_depth": 5,
    "n_estimators": 500,
    "max_samples": 0.9,
    "random_state": 42,
}
x_col = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
test_col(x_col, params2=p1)

Decision Tree
params. {'max_depth': 15, 'random_state': 42}
x_col ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
TN 98 FP 21 FN 17 TP 42
f1: 0.7881919353645269
---------------
Random Forest
params. {'max_depth': 5, 'n_estimators': 500, 'max_samples': 0.9, 'random_state': 42}
x_col ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
TN 111 FP 8 FN 19 TP 40
f1: 0.8438681745779717


In [21]:
p2 = {
    "max_depth": 3,
    "n_estimators": 100,
    "max_samples": 0.9,
    "random_state": 42,
}
x_col = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
test_col(x_col, params2=p2)

Decision Tree
params. {'max_depth': 15, 'random_state': 42}
x_col ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
TN 98 FP 21 FN 17 TP 42
f1: 0.7881919353645269
---------------
Random Forest
params. {'max_depth': 3, 'n_estimators': 100, 'max_samples': 0.9, 'random_state': 42}
x_col ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
TN 109 FP 10 FN 21 TP 38
f1: 0.8207375337747082
